In [11]:
import pandas as pd
from math import ceil, floor
import matplotlib.pyplot as plt
import seaborn as sns
import json
import matplotlib
from matplotlib import ticker
import pickle
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pickle
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora
from gensim.utils import simple_preprocess
import pandas as pd

pyLDAvis.enable_notebook()
%matplotlib inline

/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
data = pd.read_csv("../components/artifacts/append_topics.csv", sep = "\t", index_col = 0)
data["Full Text"] = data["Full Text"].fillna("")
data = data[data["Thread Entry Type"] != "share"]

/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [21]:
topic_score_cols = [x for x in data.columns if 'topic' in x]
topic_data = data[['Date','Author','Page Type','Account Type','Full Text'] + topic_score_cols]
topic_data['Topic'] = topic_data[topic_score_cols].idxmax(axis=1)
topic_data['Topic_score'] = topic_data[topic_score_cols].max(axis=1)

# topic_data.head()

/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-21-610c05730611>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topic_data['Topic'] = topic_data[topic_score_cols].idxmax(axis=1)
<ipython-input-21-610c05730611>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [24]:
topic_data['full_text_len'] = topic_data['Full Text'].apply(lambda x: len(x.split()))
# topic_data['full_text_len'].describe()
topic_data = topic_data[topic_data['full_text_len']>7]

/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-24-ade2c20aa75c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topic_data['full_text_len'] = topic_data['Full Text'].apply(lambda x: len(x.split()))


In [131]:
topic_data[(topic_data['Topic'] == 'topic_10') & (topic_data['Topic_score'] > 0.9)]['Full Text'].sample(20).tolist()

/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['matt lauer appearance krewe etat mardi gras soon skewer bad behavior',
 'comey comeyday lordy shook world create lasting chang',
 'thank sensaragelser takes incredible courage stand room capital said powerfully true leader inspiration timesup',
 'countless males aim asking nudes sending dick pics chatting counts',
 'campaign joe biden movement mobilizes butnothim opinion biden blunder wall street journal',
 'astonishes wake tirico essentially allowed skate smh ryanseacrest oscars',
 'im loudmouth lime im woman defends notme weinstein girlboss',
 'anahitaseda proud announce proud announce got gig considered promote dnc',
 'theetrain solarstormwind mindfuror madness occulturalism punkassaudrey thesharpedge thespeaker qspiracy louismcfadden thecolor jarue pitdogman gre cordicon vestonash inthematrixxx jordan_sather_ right chest pic end',
 'womans red gray short sleeve shirt bluewave gotv electionsmatter fbr resist indivisible resistance fbrparty impeach nevertrump notmypresident trumpru

In [69]:
for topic in topic_score_cols:
    print(f'TOPIC {topic}')
    data = topic_data[(topic_data['Topic'] == topic) & (topic_data['Topic_score'] > 0.95)]['Full Text'].sample(30).tolist() 
    print('\n\n'.join(data))
    print('-'*30)

/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TOPIC topic_1
positiveint yes living rent free head wish died kevin spaceys scandal

obamagate jail lyin mccabe obama rosenstein godbless potus trump pleasestand maga foxnews cnn msnbc mtv espn nfl nba pga nyt nypost wapo wsj politico gop dems resist daca oscars superbowl patriots eagles gma theview today morningjoe

bill lamb fox vp gm erica hill rodriguez journalist news director knutsenkris ouch truthhurts thats laurensivan

seattle tech peeps especially men sounds like great event womenintech wit allies

michelle williams walks golden globes carpet founder tarana burke tears eyes

bill cosby sentenced years prison celebrity trial era

im sleep mind way movement started open trauma abundance abusers hundreds honestly theyre contained group exploitation mind

electricblueyt know created black woman years ago probably didnt black person invented oxygen yall

foxnews senfeinstein senatorcollins chriscoons maziehirono senblumenthal senschumer demand fbi reopen background investigation b